In [1]:
# unset PYTHONPATH first
from ko_text import *
from ko_crawler import *

In [2]:
nlp = NLP()

# 불러오기

In [3]:
token_df = pd.read_csv('Data/meta_morphs_final.csv', encoding = 'utf-8')

# 용량을 줄이기 위해 '단어 단어' 꼴로 묶어둔 token을 ['단어', '단어'] 꼴로 풀기
token_df['Token'] = [token.split() for token in token_df['Token']]

In [4]:
token_df.head()

,Section,Text,Token,Num of Tokens
0,financial,\n\n\n텀블벅에서 크라우드 펀딩이 이뤄지고 있는 `아침달 시집`.\n\n ...,"[텀블벅, 크라, 우드, 펀딩, 이뤄지고, 아침, 시집, 많지, 않은, 금액, 으로...",263
1,economy,\n\n\n[사진 제공: 연합뉴스]\n\n 유류...,"[유류, 인하, 국제, 유가, 급락, 입어, 국내, 휘발유, 경유, 하락, 특히, ...",166
2,financial,부득이한 사정으로 매월 내는 보험료가 부담이 될 때 계약은 그대로 유지하면서 보험...,"[부득이, 사정, 매월, 내는, 보험료, 부담, 계약, 그대로, 유지, 보험료, 부...",314
3,estate,한때 `미분양의 늪`으로 통하던 경기도 파주시 부동산 시장이 달라지고 있다. 지난해...,"[한때, 미분, 하던, 경기도, 파주시, 부동산, 시장, 달라지고, 분양, 파주, ...",165
4,economy,\n\n\n인디고뱅크의 `미키인서울` 컬래버 맨투맨 <사진제공=월트디즈니코리아>\...,"[인디고, 뱅크, 미키, 서울, 컬래버, 투맨, 월트디즈니, 사의, 마스코트, 미키...",196


In [5]:
token_df.shape

(41418, 4)

# Train Test Split

In [6]:
train_size = round(len(token_df) * 0.8)
np.random.seed(0)
train_index_ls = np.random.choice(token_df.index, train_size, replace = False)
test_index_ls = [x for x in token_df.index if not x in train_index_ls]

In [7]:
train_df = token_df.loc[train_index_ls]
train_df.shape

(33134, 4)

In [8]:
test_df = token_df.loc[test_index_ls]
test_df.shape

(8284, 4)

In [9]:
Counter(train_df['Section'])

Counter({'bio & tech': 1739,
         'business': 4863,
         'culture & art': 4102,
         'economy': 2608,
         'estate': 3932,
         'financial': 746,
         'it': 1742,
         'politics': 3775,
         'society': 3356,
         'stock': 2508,
         'world': 3763})

In [10]:
Counter(test_df['Section'])

Counter({'bio & tech': 419,
         'business': 1220,
         'culture & art': 1039,
         'economy': 613,
         'estate': 977,
         'financial': 181,
         'it': 419,
         'politics': 903,
         'society': 873,
         'stock': 665,
         'world': 975})

# sampling for training classfier

In [11]:
# 한 label마다 학습할 단어의 수
train_batch_size = 1000
test_batch_size = 100

train_token_ls_split, train_tag_ls_split = nlp.oversample_batch(train_df['Token'], train_df['Section'], train_batch_size)
test_token_ls_split, test_tag_ls_split =  nlp.undersample_batch(test_df['Token'],test_df['Section'], test_batch_size)

In [12]:
print(len(train_token_ls_split))
Counter(train_tag_ls_split)

11000


Counter({'bio & tech': 1000,
         'business': 1000,
         'culture & art': 1000,
         'economy': 1000,
         'estate': 1000,
         'financial': 1000,
         'it': 1000,
         'politics': 1000,
         'society': 1000,
         'stock': 1000,
         'world': 1000})

<br>


# 잘 뽑혔는지 확인

In [ ]:
' '.join(train_token_ls_split[0])

In [14]:
train_tag_ls_split[0]

'it'

# Doc2Vec Parameter 튜닝

## **set ALPHA as default**

In [15]:
token_ls = train_df['Token'].tolist() + test_df['Token'].tolist()
label_ls = train_df['Section'].tolist() + test_df['Section'].tolist()

In [16]:
len(token_ls), len(label_ls)

(41418, 41418)

In [ ]:
%%time

result_dict = {
                'corpus_count' : [],
               'min_count' : [],
               'vector_size' : [],
               'window' : [],
               'n_epochs' : [],
               'accuracy' : [],
               'sample' : [],
               'dm' : [],
              }

testing_section_ls = ['경제','기업','사회','국제','부동산','증권','정치','IT과학','문화']

# 하이퍼 파라미터 튜닝 작업 수행

for dm in [1]:
    for doc2vec_size in ['']:
        if doc2vec_size == '':
            x_split, y_split = token_ls, label_ls

        for sample in [1e-04, 1e-05, 1e-06]:
            for min_count in [1, 5, 15, 50]:
                for vector_size in [100,300]:
                    for window in [5,15]:
                        for n_epochs in [10]:

                            # Doc2Vec 모델 생성
                            nlp.make_Doc2Vec_model(dm = dm,
                                                   min_count = min_count,
                                                   sample = sample,
                                                   vector_size = vector_size,
                                                   window = window,
                                                   dm_mean = 0,
                                                   dm_concat = 0)

                            nlp.build_and_train_Doc2Vec_model(x_split,
                                                              y_split,
                                                              n_epochs = n_epochs)


                            model_name = 'Doc2Vec_dm=%s&cc=%s&vs=%s&win=%s&min=%s&sample=%s&epochs=%s'%(\
                                                                                                                 nlp.Doc2Vec_model.dm,
                                                                                                                   nlp.Doc2Vec_model.corpus_count,
                                                                                                                   nlp.Doc2Vec_model.vector_size,
                                                                                                                   nlp.Doc2Vec_model.window,
                                                                                                                   nlp.Doc2Vec_model.min_count,
                                                                                                                   nlp.Doc2Vec_model.sample,
                                                                                                                   nlp.Doc2Vec_model.epochs)
                            # Doc2Vec 모델 저장
                            nlp.Doc2Vec_model.save('Doc2Vec_model/'+model_name)

                            '''
                            X =nlp.infer_vectors_with_Doc2Vec(train_token_ls_split)

                            from sklearn.decomposition import PCA

                            pca = PCA(n_components=2)
                            X_pca = pca.fit_transform(X)
                            scatter_df = pd.DataFrame(X_pca,
                                                      index = train_tag_ls_split,
                                                      columns = ['x','y'])

                            plt.figure(figsize = (15, 15))

                            for section in testing_section_ls:
                                temp_df = scatter_df[scatter_df.index == section]
                                plt.scatter(temp_df['x'].values, temp_df['y'].values, label = section, c = np.random.rand(3,))

                            plt.legend(loc = 'best')
                            #plt.savefig('Doc2Vec_model/images/'+model_name)
                            #plt.imshow()

                            '''
                            
                            # clf를 각 레이블별 1000개씩 학습, 
                            X_train = nlp.infer_vectors_with_Doc2Vec(train_token_ls_split)
                            y_train = train_tag_ls_split

                            X_test = nlp.infer_vectors_with_Doc2Vec(test_token_ls_split)
                            y_test = test_tag_ls_split


                            clf = LogisticRegression(solver = 'sag',
                                                     multi_class = 'multinomial')


                            clf.fit(X_train, y_train)
                            y_pred = clf.predict(X_test)
                            
                            result_dict['dm'].append(nlp.Doc2Vec_model.dm)
                            result_dict['corpus_count'].append(nlp.Doc2Vec_model.corpus_count)
                            result_dict['min_count'].append(nlp.Doc2Vec_model.min_count)
                            result_dict['vector_size'].append(nlp.Doc2Vec_model.vector_size)
                            result_dict['window'].append(nlp.Doc2Vec_model.window)
                            result_dict['n_epochs'].append(nlp.Doc2Vec_model.epochs)
                            result_dict['sample'].append(nlp.Doc2Vec_model.sample)
                            result_dict['accuracy'].append(accuracy_score(y_pred, y_test))

                            print(accuracy_score(y_pred, y_test))


            pd.DataFrame(result_dict).to_csv('Parameter_tuning_result.csv', index = False)

2018-12-04 16:11:03,748 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-12-04 16:11:03,807 : INFO : collecting all words and their counts
2018-12-04 16:11:03,808 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-12-04 16:11:04,140 : INFO : PROGRESS: at example #10000, processed 1884311 words (5681463/s), 81597 word types, 11 tags
2018-12-04 16:11:04,486 : INFO : PROGRESS: at example #20000, processed 3762527 words (5436144/s), 110172 word types, 11 tags
2018-12-04 16:11:04,875 : INFO : PROGRESS: at example #30000, processed 5590210 words (4711053/s), 130122 word types, 11 tags
2018-12-04 16:11:05,223 : INFO : PROGRESS: at example #40000, processed 7430810 words (5298256/s), 143472 word types, 11 tags
2018-12-04 16:11:05,287 : INFO : collected 148759 word types and 11 unique tags from a corpus of 41418 examples and 7723626 words
2018-12-04 16:11:05,287 : INFO : Loading a fresh vocabulary
2018-12-04 16:11:06,064 :

0.84


2018-12-04 16:14:07,220 : INFO : PROGRESS: at example #10000, processed 1884311 words (6595444/s), 81597 word types, 11 tags
2018-12-04 16:14:07,538 : INFO : PROGRESS: at example #20000, processed 3762527 words (5917540/s), 110172 word types, 11 tags
2018-12-04 16:14:07,850 : INFO : PROGRESS: at example #30000, processed 5590210 words (5868573/s), 130122 word types, 11 tags
2018-12-04 16:14:08,156 : INFO : PROGRESS: at example #40000, processed 7430810 words (6021930/s), 143472 word types, 11 tags
2018-12-04 16:14:08,209 : INFO : collected 148759 word types and 11 unique tags from a corpus of 41418 examples and 7723626 words
2018-12-04 16:14:08,210 : INFO : Loading a fresh vocabulary
2018-12-04 16:14:08,955 : INFO : effective_min_count=1 retains 148759 unique words (100% of original 148759, drops 0)
2018-12-04 16:14:08,956 : INFO : effective_min_count=1 leaves 7723626 word corpus (100% of original 7723626, drops 0)
2018-12-04 16:14:09,300 : INFO : deleting the raw counts dictionary of 

In [13]:
nlp.Doc2Vec_model.docvecs.vectors_docs.shape

(9, 300)

<br>
